# Занятие 3. Исключения, контекстные менеджеры и пакеты

## Исключения

Исключения - это объекты, вызываемые для остановки работы программы с сообщением о том, что произошла ошибка

In [ ]:
1 / 0

ZeroDivisionError: division by zero

In [ ]:
positive_number = int(input("Введите положительное число: "))
if positive_number <= 0:
    raise Exception("Я же просил положительное число :(")
print(positive_number ** (1 / 2))

Введите положительное число:  16


4.0


### Классы исключений 

Исключения в Python являются классами ошибок. В Python есть много стандартных исключений. Они имеют определённую иерархию за счёт механизма наследования классов. Все исключения являются потомками базового класса BaseException

В документации Python версии 3.10.8 приводится следующее дерево иерархии стандартных исключений:
![image.png](attachment:5979d1bb-fbaa-43b2-9742-960df7b13851.png)

### try-except-finally

С помощью конструкции try-except-finally можно отлавливать и обрабатывать исключения в формате:

```python
try:
    <code>
except <exception1> as <err name>:
    <code>
except <exception2> as <err name>:
    <code>
else:
    <code>  # выполняется, если не вызвано исключение в блоке try
finally:
    <code>  # выполняется всегда
```

In [ ]:
cities = {"Saint-Petersburg": 2, "Kazan": 2, "Suzdal": 1}
try:
    city = input()
    print(f"В городе {city} я был {cities[city]} раз")
except KeyError as err:
    print(f"В городе {city} я еще не был")

In [ ]:
try:
    positive_number = 1 / int(input("Введите положительное число: "))  # а что если 0 или строка?
    print(positive_number ** (1 / 2))
except ZeroDivisionError:
    print("Обработка ошибки деления на 0...")
except ValueError:
    print("Обработка ошибки ввода строки...")
else:
    print("Ошибка не произошла")
finally:  
    print("А я выполнюсь в любом случае")

Введите положительное число:  2


0.7071067811865476
Ошибка не произошла
А я выполнюсь в любом случае


In [ ]:
try:
    positive_number = 1 / int(input("Введите положительное число: "))  # а что если 0 или строка?
    print(positive_number ** (1 / 2))
except BaseException:
    print("Неизвестная ошибка...")
except ZeroDivisionError:
    print("Обработка ошибки деления на 0...")
except ValueError:
    print("Обработка ошибки ввода строки...")

Введите положительное число:  qw


Неизвестная ошибка...


In [ ]:
try:
    positive_number = 1 / int(input("Введите положительное число: "))  # а что если 0 или строка?
    print(positive_number ** (1 / 2))
except ZeroDivisionError as err:
    print(f"Произошла ошибка:\n{err.__class__.__name__}: {err}")
except ValueError as err:
    print(f"Произошла ошибка:\n{err.__class__.__name__}: {err}")
except BaseException as err:
    print(f"Произошла ошибка:\n{err.__class__.__name__}: {err}")

Введите положительное число:  0


Произошла ошибка:
ZeroDivisionError: division by zero


### Собственные исключения

In [ ]:
class NonAdminError(AttributeError):
    def __str__(self):
        return "У вас нет доступа к этому функционалу"

admins = {"Peter", "Yaroslav", "Artem", "Andrew", "Stepan"}
person = input("Введите имя администратора: ")
if person not in admins:
    raise NonAdminError
print(person, "- администратор курса")

Введите имя администратора:  Peter


Peter - администратор курса


## Context manager

>«Контекстные менеджеры в Python — это удивительный механизм, который позволяет гарантировать корректное управление ресурсами и обеспечивать безопасное выполнение кода.» — Гвидо ван Россум, великодушный пожизненный диктатор Python.

### Работа с файлами

In [165]:
def divide(a: float, b: float) -> float:
    try:
        return a / b
    except ZeroDivisionError:
        return float("inf")


divide(1, 0)

inf

In [167]:
file = open("example.txt", "r")
try:
    # Действия с файлом
    content = file.read()
    print(content)
finally:
    file.close()

Hello\nWorld!



In [169]:
with open("example.txt", "r") as file:
    content = file.read()
    print(content)
# --> file.close()

Hello\nWorld!



### Работа с бд

In [180]:
import sqlite3

# Пример работы с SQLite базой данных
with sqlite3.connect("example.db") as conn:
    cursor = conn.cursor()
    # Выполнение операций с базой данных
    cursor.execute("CREATE TABLE IF NOT EXISTS user(id INTEGER PRIMARY KEY, name TEXT)")
    cursor.execute("INSERT INTO user(name) VALUES ('teadove'), ('tainella')")
    cursor.execute("SELECT * FROM user")
    result = cursor.fetchall()
    print(result)

[(1, 'a'), (2, 'teadove'), (3, 'tainella'), (4, 'teadove'), (5, 'tainella')]


### Свой контекстный менеджер

Контекстный менеджер в Python должен содержать методы `__enter__` и `__exit__`.

Метод `__enter__` выполняется перед выполнением блока кода внутри оператора with. Он может выполнять какие-либо подготовительные действия или возвращать значение, которое будет связано с переменной после ключевого слова as.

Метод `__exit__` вызывается после завершения выполнения блока кода with. Он используется для выполнения завершающих действий, таких как освобождение ресурсов, обработка исключений или выполнение финализирующих операций.

In [181]:
class ErrorMutter:
    def __enter__(self):
        print("enter method called")
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print("exit method called")


with ContextManager() as manager:
    print("with statement block")

NameError: name 'ContextManager' is not defined

In [ ]:
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None

    def __enter__(self):
        self.file = open(self.filename, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.file.close()


# загрузка файла
with FileManager("example.txt", "r") as f:
    print(f.encoding)

print(f.closed)

## Доп. материалы

[Статья про декораторы и генераторы от Яндекса](https://academy.yandex.ru/handbook/python/article/rekursiya-dekoratory-generatory)

[Некоторые хитрости при работе с итераторами](https://habr.com/ru/articles/488112/)

[Подробнее про context manager и его применения](https://realpython.com/python-with-statement/)

Для продвинутых: 

[Интересная статья про асинхронность: контекстные менеджеры и не только](https://habr.com/ru/companies/wunderfund/articles/711012/)